In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 21.6 MB/s 
     |████████████████████████████████| 86 kB 2.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.5 MB/s 


In [3]:
!pip3 install mvlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.5 MB/s 
     |████████████████████████████████| 38.1 MB 1.7 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('drive/MyDrive/William_2022/DATASETS/trans_encoder_2')

In [103]:
!python3 text_Encoder.py

[ Epoch 0 ]
[1.0, 0.045662100456621]
[0.11814345991561181, 1.0]
acc_SP is : 0.045662100456621
pre_i is : [1.0, 0.045662100456621]
rec_i is : [0.11814345991561181, 1.0]
F1_i is : [0.21132075471698114, 0.08733624454148473]
    - [Info] The checkpoint file has been updated.
  - (Training)  loss:  0.35933, accuracy: 25.556 %, elapse: 0.062 min
train_cm: [[539 524]
 [532 565]]
  - (Validation)  loss:  0.47560, accuracy: 7.692 %, elapse: 0.063 min
valid_cm: [[ 28 209]
 [  0  10]]
[ Epoch 1 ]
[1.0, 0.04081632653061224]
[0.008438818565400843, 1.0]
acc_SP is : 0.04081632653061224
pre_i is : [1.0, 0.04081632653061224]
rec_i is : [0.008438818565400843, 1.0]
F1_i is : [0.01673640167364017, 0.07843137254901959]
  - (Training)  loss:  0.35845, accuracy: 25.833 %, elapse: 0.020 min
train_cm: [[433 601]
 [443 683]]
  - (Validation)  loss:  0.38048, accuracy: 2.429 %, elapse: 0.020 min
valid_cm: [[  2 235]
 [  0  10]]
[ Epoch 2 ]
text_Encoder.py:53: RuntimeWarning: invalid value encountered in long_sca

# MultiModal

In [39]:
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import pandas as pd
import numpy as np
from model_new import Transformer, Transformer2, Encoder
from config import *
from imblearn.over_sampling import RandomOverSampler
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from tqdm import tqdm
from optim_new import ScheduledOptim
from torch.optim import Adam
from sklearn.metrics import confusion_matrix
from CCA import DeepCCA, cca_loss
from scipy.stats import wasserstein_distance
from mvlearn.embed import DCCA
from mvlearn.datasets import make_gaussian_mixture
from mvlearn.plotting import crossviews_plot
from dataset_new import Fusion

In [7]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

class EEGDataset(Dataset):
    def __init__(self, signal, label):

        self._signal = torch.FloatTensor(signal)
        self._label = torch.LongTensor(label)


    @property
    def n_insts(self):
        ''' Property for dataset size '''
        return len(self._label)

    @property
    def sig_len(self):
        return self._signal.shape[1]

    def __len__(self):
        return self.n_insts

    def __getitem__(self, idx):
        return self._signal[idx], self._label[idx]

class TextDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  @property
  def n_insts(self):
    return len(self.labels)

  @property
  def text_len(self):
    return 32

  def __len__(self):
    return self.n_insts

  def __getitem__(self, item):
    text = str(self.texts[item])
    label = self.labels[item]

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length = True,
      return_attention_mask=True
      #return_tensors='pt',
    )
    return torch.FloatTensor(encoding['input_ids']).flatten(), torch.tensor(label, dtype=torch.long)



Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
# --- Preprocess
df = pd.read_csv('df.csv')

X = df.drop([emotion], axis = 1)
y= df[[emotion]]

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 2, test_size = 0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, random_state= 2, test_size = 0.5)
df_test = pd.concat([X_test, y_test], axis = 1)
df_train = pd.concat([X_train, y_train], axis = 1)
df_val = pd.concat([X_val, y_val], axis = 1)

df_train_text = df_train[[emotion, 'new_words']]
df_train_eeg = df_train[eeg]

df_val_text = df_val[[emotion, 'new_words']]
df_val_eeg = df_val[eeg]

df_test_text = df_test[[emotion, 'new_words']]
df_test_eeg = df_test[eeg]

# --- Save CSV
df_train_text.to_csv('df_train_text.csv', header = None, index = False, index_label = False)
df_train_eeg.to_csv('df_train_eeg.csv', header = None, index = False, index_label = False)

df_val_text.to_csv('df_val_text.csv', header = None, index = False, index_label = False)
df_val_eeg.to_csv('df_val_eeg.csv', header = None, index = False, index_label=False)


df_test_text.to_csv('df_test_text.csv', header = None, index = False, index_label = False)
df_test_eeg.to_csv('df_test_eeg.csv', header = None, index = False, index_label=False)

# --- Load CSV
df_train_text = pd.read_csv('df_train_text.csv', header = None).values
df_train_eeg = pd.read_csv('df_train_eeg.csv', header = None).values

df_val_text= pd.read_csv('df_val_text.csv', header = None).values
df_val_eeg = pd.read_csv('df_val_eeg.csv', header = None).values

df_test_text= pd.read_csv('df_test_text.csv', header = None).values
df_test_eeg = pd.read_csv('df_test_eeg.csv', header = None).values

# --- RandomOverSampling
# Text

X_text_train = df_train_text[:, 1:]
y_text_train = df_train_text[:, 0]
y_text_train = y_text_train.astype('int')

ros = RandomOverSampler(random_state=2)
X_resampled_text, y_resampled_text = ros.fit_resample(X_text_train, y_text_train)

# print(df_)
# EEG

X_eeg_train = df_train_eeg[:, 1:]
y_eeg_train = df_train_eeg[:, 0]

ros = RandomOverSampler(random_state=2)
X_resampled_eeg, y_resampled_eeg = ros.fit_resample(X_eeg_train, y_eeg_train)



# --- Text
train_text = TextDataset(
    texts = X_resampled_text,
    labels = y_resampled_text,
    tokenizer = tokenizer,
    max_len = MAX_LEN
)
val_text = TextDataset(
    texts = df_val_text[:, 1:],
    labels = df_val_text[:, 0],
    tokenizer = tokenizer,
    max_len = MAX_LEN
)

test_text = TextDataset(
  texts = df_test_text[:, 1:],
  labels = df_test_text[:, 0],
  tokenizer = tokenizer,
  max_len = MAX_LEN

)
train_loader_text = DataLoader(dataset=train_text,
                          batch_size=batch_size,
                          num_workers=2
                          )
valid_loader_text = DataLoader(dataset=val_text,
                          batch_size=batch_size,
                          num_workers=2
                          )
test_loader_text = DataLoader(dataset=test_text,
                          batch_size=batch_size,
                          num_workers=2
                          )
# --- EEG
train_eeg = EEGDataset(
    signal = X_resampled_eeg,
    label = y_resampled_eeg
)

val_eeg = EEGDataset(
    signal = df_val_eeg[:, 1:],
    label = df_val_eeg[:, 0]
)

test_eeg = EEGDataset(
  signal = df_test_eeg[:, 1:],
  label = df_test_eeg[:, 0]
)
# --- Dataloader EEG
train_loader_eeg = DataLoader(dataset=train_eeg,
                          batch_size=batch_size,
                          num_workers=2
                          )
valid_loader_eeg = DataLoader(dataset=val_eeg,
                          batch_size=batch_size,
                          num_workers=2)

test_loader_eeg = DataLoader(dataset=test_eeg,
                          batch_size=batch_size,
                          num_workers=2)

In [ ]:
from torch.utils.data.sampler import BatchSampler
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, dataset, n_classes, n_samples):
        loader = DataLoader(dataset)
        self.labels_list = []
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [ ]:
y_text_train = y_text_train.astype('int')

In [ ]:
target = df_train_text[:, 0].astype('int')

In [ ]:
class_sample_count = np.unique(target, return_counts=True)[1]

weight = 1. / class_sample_count
samples_weight = weight[target]

In [ ]:
class_sample_count = np.unique(target, return_counts=True)[1]

weight = 1. / class_sample_count
samples_weight = weight[target]
samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [9]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [88]:
model1 = Transformer(device = device, d_feature=train_text.text_len, d_model=16, d_inner=32,
                            n_layers=2, n_head=4, d_k=64, d_v=64, dropout=dropout,
                            class_num=class_num)

In [61]:
model2 = Transformer2(device=device, d_feature=train_eeg.sig_len, d_model=d_model, d_inner=d_inner,
                            n_layers=num_layers, n_head=num_heads, d_k=64, d_v=64, dropout=dropout, class_num=class_num)

In [62]:
class Fusion(nn.Module):
  def __init__(self, model1, model2, d_feature = 40, d_model = 16, class_num = class_num):
    super(Fusion, self).__init__()
    self.model1 = model1
    self.model2 = model2
    # self.classifier = nn.Linear(4, class_num)
    self.linear1_cov = nn.Conv1d(d_feature, 1, kernel_size=1)
    self.linear1_linear = nn.Linear(d_model, class_num)
    # self.linear2_cov = nn.Conv1d(d_model, 1, kernel_size=1)
    # self.linear2_linear = nn.Linear(d_feature, class_num)

  def forward(self, x1, x2):
    x1 = self.model1(x1)
    x2 = self.model2(x2)
    x = torch.cat((x1, x2), dim = 1)
    out = self.linear1_cov(x)
    out = self.linear1_linear(out)
    return out, x1, x2, x

In [89]:
model1 = nn.DataParallel(model1)
model2 = nn.DataParallel(model2)

In [94]:
chkpt1 = torch.load(torchload, map_location = 'cuda')
chkpt2 = torch.load(torchload2, map_location = 'cuda')


In [95]:
model1

DataParallel(
  (module): Transformer(
    (encoder): Encoder(
      (src_word_emb): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
      (position_enc): Embedding(33, 16)
      (layer_stack): ModuleList(
        (0): EncoderLayer(
          (slf_attn): MultiHeadAttention(
            (w_qs): Linear(in_features=16, out_features=256, bias=True)
            (w_ks): Linear(in_features=16, out_features=256, bias=True)
            (w_vs): Linear(in_features=16, out_features=256, bias=True)
            (attention): SDPAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (softmax): Softmax(dim=0)
              (BN): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (layer_norm): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
            (fc): Linear(in_features=256, out_features=16, bias=True)
            (dropout): Dropout(p=0.3, inplace=False)
          )
          (pos_ffn): PositionwiseFee

In [ ]:
use_all_singular_values = False
outdim_size = class_num

In [83]:
model = Fusion(model1, model2).to(device)

In [90]:
def cal_loss(pred, label, device):

    cnt_per_class = np.zeros(2)

    loss = F.cross_entropy(pred, label, reduction='sum')
    pred = pred.max(1)[1]
    n_correct = pred.eq(label).sum().item()
    cnt_per_class = [cnt_per_class[j] + pred.eq(j).sum().item() for j in range(class_num)]
    return loss, n_correct, cnt_per_class

In [91]:
optimizer = ScheduledOptim(
            Adam(filter(lambda x: x.requires_grad, model1.parameters()),
                 betas=(0.9, 0.98), eps=1e-4), d_model, warm_steps)
        
optimizer2 = ScheduledOptim(
            Adam(filter(lambda x: x.requires_grad, model2.parameters()),
                 betas=(0.9, 0.98), eps=1e-4), d_model, warm_steps)

In [93]:
model1.train()
all_labels = []
all_res = []
all_res2 = []
total_loss = 0
total_correct = 0
#cnt_per_class = np.zeros(class_num)


dataloader_iterator = iter(train_loader_text)

for i, data1 in enumerate(train_loader_eeg):

  try:
      data2 = next(dataloader_iterator)
  except StopIteration:
      dataloader_iterator = iter(train_loader_text)
      data2 = next(dataloader_iterator)

  sig1, label1 = map(lambda x: x.to(device), data2)
  sig2, label2 = map(lambda x: x.to(device), data1)

  
  optimizer.zero_grad()
  out = model1(sig1)
  print(out)
  break
  all_labels.extend(label1.cpu().numpy())
  all_res.extend(pred1.max(1)[1].cpu().numpy())
  all_res2.extend(pred2.max(1)[1].cpu().numpy())
  loss, n_correct1 = cal_loss(pred1, label1, pred2, device)
  # print(loss)
  break
  
  
  loss.backward()
  optimizer.step_and_update_lr()
  total_loss += loss.item()
  total_correct += (n_correct1)


  cm = confusion_matrix(all_labels, all_res)
  cm2 = confusion_matrix(all_labels, all_res2)
  

# train_loss = total_loss / (total_num + total_num2)
# train_acc = total_correct / (total_num + total_num2)



/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

tensor([[ 0.3766,  0.6605],
        [ 0.6045,  0.2237],
        [ 0.2652,  0.0370],
        [ 0.4254,  0.1651],
        [ 0.2152, -0.0459],
        [-0.0949,  0.0811],
        [ 0.6418,  0.3461],
        [ 0.7179,  0.7258],
        [ 0.1896,  0.5325],
        [ 0.8326,  0.2649],
        [ 0.2779,  0.3462],
        [ 0.9533,  0.4710],
        [ 0.6114, -0.1127],
        [ 0.0500, -0.0333],
        [ 0.1028,  0.2828],
        [ 0.3871,  0.2907],
        [ 0.3782,  0.1505],
        [-0.1609,  0.1528],
        [ 0.1413,  0.0638],
        [ 0.1321, -0.2834],
        [ 0.8339, -0.2071],
        [ 0.5400, -0.5251],
        [ 0.7196, -0.0309],
        [ 0.1788,  0.4507],
        [ 0.7174,  0.0455],
        [ 0.2799,  0.3270],
        [-0.1203,  0.0803],
        [ 0.2433,  0.4512],
        [ 0.3626,  0.1326],
        [ 0.3562,  0.0721],
        [ 0.6156,  0.6227],
        [ 0.4385,  0.4387]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
'''
H1 = text
H2 = eeg

sig1 = text
sig2 = eeg

o1 = o2 = size of text

m = text row length

H1bar = at each row take the mean
'''
r1= 1e-3
r2 = 1e-3
eps = 1e-9

H1, H2 = sig1.t(), sig2.t()

o1 = H1.size(0)
o2 = H2.size(0)
m = H1.size(1)

H1bar = H1 - H1.mean(dim = 1).unsqueeze(dim=1)
H2bar = H2 - H2.mean(dim =1).unsqueeze(dim=1)

SigmaHat12 = (1.0/(m-1)) * torch.matmul(H1bar, H2bar.t())
SigmaHat11 = (1.0 / (m-1)) * torch.matmul(H1bar, H1bar.t()) + r1 * torch.eye(o1, device = device)
SigmaHat22 = (1.0 / (m-1)) * torch.matmul(H2bar, H2bar.t()) + r2 * torch.eye(o2, device = device)


[D1, V1] = torch.symeig(SigmaHat11, eigenvectors = True)
[D2, V2] = torch.symeig(SigmaHat22, eigenvectors = True)

posInd1 = torch.gt(D1, eps).nonzero()[:, 0]
D1 = D1[posInd1]
V1 = V1[:, posInd1]
posInd2 = torch.gt(D2, eps).nonzero()[:, 0]
D2 = D2[posInd2]
V2 = V2[:, posInd2]



SigmaHat11RootInv = torch.matmul(
    torch.matmul(V1, torch.diag(D1 ** -0.5)), V1.t())
SigmaHat22RootInv = torch.matmul(
    torch.matmul(V2, torch.diag(D2 ** -0.5)), V2.t())

Tval = torch.matmul(torch.matmul(SigmaHat11RootInv,
                                  SigmaHat12), SigmaHat22RootInv)


trace_TT = torch.matmul(Tval.t(), Tval)
trace_TT = torch.add(trace_TT, (torch.eye(trace_TT.shape[0])*r1).to(device)) # regularization for more stability
U, V = torch.symeig(trace_TT, eigenvectors=True)
U = torch.where(U>eps, U, (torch.ones(U.shape).float()*eps).to(device))
U = U.topk(outdim_size)[0]
corr = torch.sum(torch.sqrt(U))

In [ ]:
corr

tensor(1.9993)